In [3]:
import sqlite3
conn = sqlite3.connect('demo.db') 
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Bang1 (
        A REAL,
        B REAL
    );
''')
cursor.executemany('''
    INSERT INTO Bang1 (A, B)
    VALUES (?, ?);
''', [
    (1, 2),
    (2, 3),
    (3, 4),
    (4, 5),
    (5, 6)])
conn.commit()
cursor.execute('''
    SELECT 
        (COUNT(*) * SUM(A * B) - SUM(A) * SUM(B)) / 
        (
            (COUNT(*) * SUM(A * A) - SUM(A)*SUM(A)) * 
            (COUNT(*) * SUM(B * B) - SUM(B)*SUM(B))
        ) AS correlation_coefficient
    FROM 
        Bang1;
''')
correlation_coefficient = cursor.fetchone()[0]
print(f'Hệ số tương quan Pearson giữa A và B là: {correlation_coefficient}')

Hệ số tương quan Pearson giữa A và B là: 0.0022222222222222222


In [4]:
import pandas as pd
import scipy.stats as stats
cursor.execute('''
    CREATE TABLE car_scores (
        day TEXT,
        model TEXT,
        score REAL
    )
''')
data = [
    ('Day 1', 'A', 8), ('Day 1', 'B', 9), ('Day 1', 'C', 7),
    ('Day 2', 'A', 7.5), ('Day 2', 'B', 8.5), ('Day 2', 'C', 7),
    ('Day 3', 'A', 6), ('Day 3', 'B', 7), ('Day 3', 'C', 8),
    ('Day 4', 'A', 7), ('Day 4', 'B', 6), ('Day 4', 'C', 5)]
cursor.executemany('INSERT INTO car_scores (day, model, score) VALUES (?, ?, ?)', data)
conn.commit()
df = pd.read_sql_query('''SELECT * FROM car_scores''', conn)
pivot_table = df.pivot(index='day', columns='model', values='score')
chi2_stat, p_value, dof, expected = stats.chi2_contingency(pivot_table)
print('Bảng dữ liệu:\n', pivot_table)
print('\nThống kê Chi-Square:', chi2_stat)
print('Giá trị P:', p_value)
print('Bậc tự do:', dof)
print('Tần suất kỳ vọng:\n', expected)

Bảng dữ liệu:
 model    A    B    C
day                 
Day 1  8.0  9.0  7.0
Day 2  7.5  8.5  7.0
Day 3  6.0  7.0  8.0
Day 4  7.0  6.0  5.0

Thống kê Chi-Square: 0.826439691310499
Giá trị P: 0.9913459315666546
Bậc tự do: 6
Tần suất kỳ vọng:
 [[7.95348837 8.51162791 7.53488372]
 [7.62209302 8.15697674 7.22093023]
 [6.95930233 7.44767442 6.59302326]
 [5.96511628 6.38372093 5.65116279]]


In [5]:
cursor.execute('CREATE TABLE flights (id INTEGER PRIMARY KEY, departure_time INTEGER)')
data = [(1, 830), (2, 1445), (3, 1230), (4, 700), (5, 2200)]
cursor.executemany('INSERT INTO flights (id, departure_time) VALUES (?, ?)', data)
conn.commit()
cursor.execute('''
SELECT id, departure_time,
       printf('%02d:%02d %s', 
              departure_time / 100 % 12, 
              departure_time % 100, 
              CASE WHEN departure_time / 100 >= 12 THEN 'PM' ELSE 'AM' END
       ) AS formatted_time
FROM flights
''')
for row in cursor.fetchall():
    print(row)

(1, 830, '08:30 AM')
(2, 1445, '02:45 PM')
(3, 1230, '00:30 PM')
(4, 700, '07:00 AM')
(5, 2200, '10:00 PM')


In [6]:
import numpy as np
cursor.execute('''
    CREATE TABLE IF NOT EXISTS data_table (
        id INTEGER PRIMARY KEY,
        value INTEGER
    )
''')
cursor.executemany('''
    INSERT INTO data_table (value)
    VALUES (?)
''', [(830,), (1111,), (930,), (1230,), (1100,)])
conn.commit()
cursor.execute('''
    SELECT value
    FROM data_table
''')
values = [row[0] for row in cursor.fetchall()]
median_value = np.median(values)
deviations = [abs(value - median_value) for value in values]
mad_value = np.median(deviations)
outliers = []
for i, deviation in enumerate(deviations):
    if deviation > 1.5 * mad_value:
        outliers.append((values[i], deviation))
print(f'Trung vị (Median): {median_value}')
print(f'MAD: {mad_value}')
print('Các ngoại lệ:')
for outlier in outliers:
    print(f'Value: {outlier[0]}, Absolute Deviation: {outlier[1]}')

Trung vị (Median): 1100.0
MAD: 130.0
Các ngoại lệ:
Value: 830, Absolute Deviation: 270.0


In [7]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Patient (
        id INTEGER PRIMARY KEY,
        last_name TEXT,
        weight REAL,
        height REAL
    )
''')
cursor.executemany('''
    INSERT INTO Patient (last_name, weight, height)
    VALUES (?, ?, ?)
''', [
    ('Smith', 70, 170),
    ('Johnson', 70, 175),
    ('Smith', 70, 165),
    ('Johnson', 80, 180),
    ('Smith', 72, 168)
])
conn.commit()
cursor.execute('''
    SELECT 
        p1.last_name AS last_name1,
        p1.weight AS weight1,
        p2.last_name AS last_name2,
        p2.weight AS weight2,
        CASE 
            WHEN p1.last_name = p2.last_name THEN 0 ELSE 1 
        END AS last_name_distance,
        CASE 
            WHEN p1.weight = p2.weight THEN 0 ELSE 1 
        END AS weight_distance,
        (CASE 
            WHEN p1.last_name = p2.last_name THEN 0 ELSE 1 
        END + CASE 
            WHEN p1.weight = p2.weight THEN 0 ELSE 1 
        END) AS total_distance
    FROM Patient p1
    CROSS JOIN Patient p2
    WHERE p1.id < p2.id;
''')
matches = cursor.fetchall()
print('So sánh các cặp người:')
for match in matches:
    last_name_distance = match[4]
    weight_distance = match[5]
    total_distance = match[6]
    print(f'{match[0]} và {match[2]} (Trọng lượng: {match[1]}, {match[3]}) => Total Distance: {total_distance}')
    if total_distance == 0:
        print(f' -> Cả hai là một người.')
    else:
        print(f' -> Cả hai không phải là một người.')
conn.close()


So sánh các cặp người:
Smith và Johnson (Trọng lượng: 70.0, 70.0) => Total Distance: 1
 -> Cả hai không phải là một người.
Smith và Smith (Trọng lượng: 70.0, 70.0) => Total Distance: 0
 -> Cả hai là một người.
Smith và Johnson (Trọng lượng: 70.0, 80.0) => Total Distance: 2
 -> Cả hai không phải là một người.
Smith và Smith (Trọng lượng: 70.0, 72.0) => Total Distance: 1
 -> Cả hai không phải là một người.
Johnson và Smith (Trọng lượng: 70.0, 70.0) => Total Distance: 1
 -> Cả hai không phải là một người.
Johnson và Johnson (Trọng lượng: 70.0, 80.0) => Total Distance: 1
 -> Cả hai không phải là một người.
Johnson và Smith (Trọng lượng: 70.0, 72.0) => Total Distance: 2
 -> Cả hai không phải là một người.
Smith và Johnson (Trọng lượng: 70.0, 80.0) => Total Distance: 2
 -> Cả hai không phải là một người.
Smith và Smith (Trọng lượng: 70.0, 72.0) => Total Distance: 1
 -> Cả hai không phải là một người.
Johnson và Smith (Trọng lượng: 80.0, 72.0) => Total Distance: 2
 -> Cả hai không phải là một